In [1]:
# import pandas as pd
# from openpyxl.styles import PatternFill
# import random
# from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus, PULP_CBC_CMD
import pandas as pd
import random
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, LpStatus, PULP_CBC_CMD
from openpyxl.styles import PatternFill


# Função para construir o DataFrame df_assigned com os resultados do solver
def build_df_assigned(teachers, classes, periods, x, teacher_id_to_name):
    results = []
    for i in teachers:
        for j in classes:
            for k in periods:
                if x[i][j][k].varValue == 1:  # Condição de atribuição: a aula foi marcada
                    results.append({'Teacher': teacher_id_to_name[i], 'Class': j, 'Period': k})
    df_assigned = pd.DataFrame(results)
    return df_assigned

# Função para mapear períodos para dias da semana e números de períodos diários
def period_to_day_period(period):
    day = (period - 1) // 10
    daily_period = (period - 1) % 10 + 1
    return day, daily_period

# Função para obter uma cor aleatória em formato RGB
def get_random_color(pastel_factor=0.5):
    return [(x + pastel_factor) / (1.0 + pastel_factor) for x in [random.uniform(0, 1.0) for _ in range(3)]]

# Função para calcular a distância entre duas cores
def color_distance(c1, c2):
    return sum([abs(x[0] - x[1]) for x in zip(c1, c2)])


# Função para gerar uma nova cor que seja visualmente distinta das cores existentes
# Crédito: https://gist.github.com/adewes/5884820
def generate_new_color(existing_colors, pastel_factor=0.5):
    max_distance = None
    best_color = None
    for _ in range(100):
        color = get_random_color(pastel_factor=pastel_factor)
        if not existing_colors:
            return color
        best_distance = min([color_distance(color, c) for c in existing_colors])
        if not max_distance or best_distance > max_distance:
            max_distance = best_distance
            best_color = color
    return best_color

# Função para converter uma cor RGB em hexadecimal
def rgb_to_hex(color):
    return ''.join(f'{int(x * 255):02X}' for x in color)

# Função para gerar a timetable colorida e salvar em Excel
def generate_timetable_excel(df_assigned, teacher_id_to_name, excel_filename):
    # Lista dos dias da semana
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    classes = ['6oA', '6oB', '7oA', '7oB', '8oA', '8oB','9oA', '9oB', '1oA', '1oB','2oA', '2oB', '3o']
    
    # Mapear períodos para dias da semana e número do período como feito anteriormente
    df_assigned['Day'], df_assigned['DailyPeriod'] = zip(*df_assigned['Period'].apply(period_to_day_period))
    df_assigned['Day'] = df_assigned['Day'].apply(lambda x: days_of_week[x])
    df_assigned['Day'] = pd.Categorical(df_assigned['Day'], categories=days_of_week, ordered=True)
    df_assigned['Class'] = pd.Categorical(df_assigned['Class'], categories=classes, ordered=True)
    
    # Organizando os dados no formato de timetable
    timetable = df_assigned.pivot_table(index=['Day', 'DailyPeriod'], columns='Class', values='Teacher', aggfunc='first', observed=False).fillna('')

    
    # Atribuição de cores aos professores
    unique_teachers = df_assigned['Teacher'].unique()
    existing_colors = []
    teacher_color_map = {}
    for teacher in unique_teachers:
        new_color = generate_new_color(existing_colors)
        existing_colors.append(new_color)
        teacher_color_map[teacher] = rgb_to_hex(new_color)

    # Exportando para Excel com cores
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        timetable.to_excel(writer, sheet_name='Timetable')
        workbook = writer.book
        worksheet = writer.sheets['Timetable']

        # Aplicando cores às células do Excel
        for row in worksheet.iter_rows(min_row=2, min_col=2):
            for cell in row:
                teacher = cell.value
                if teacher and teacher in teacher_color_map:
                    fill_color = teacher_color_map[teacher]
                    cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")

    print(f"Arquivo Excel salvo como {excel_filename}")

# Função que executa todas as etapas: constrói o DataFrame df_assigned, gera e exporta a timetable em Excel
def generate_and_export_timetable(teachers, classes, periods, x, dados, excel_filename):
    # Construir o mapeamento de ID do professor para o nome do professor
    teacher_id_to_name = dict(zip(dados['No'], dados['Professor']))

    # Construir o DataFrame df_assigned com os resultados do solver
    df_assigned = build_df_assigned(teachers, classes, periods, x, teacher_id_to_name)

    # Gerar e exportar a timetable em Excel com cores
    generate_timetable_excel(df_assigned, teacher_id_to_name, excel_filename)
def generate_and_export_timetableFO(teachers, classes, periods, x, dados, excel_filename,df_assigned):
    # Construir o mapeamento de ID do professor para o nome do professor
    teacher_id_to_name = dict(zip(dados['No'], dados['Professor']))

    # Gerar e exportar a timetable em Excel com cores
    generate_timetable_excel(df_assigned, teacher_id_to_name, excel_filename)

# Exemplo de uso:
# Supondo que você tenha teachers, classes, periods, x e dados definidos
# generate_and_export_timetable(teachers, classes, periods, x, dados, 'timetable_colored.xlsx')

def avaliar_solucao(solution):
    idle_periods = 0
    double_classes = 0
    triple_classes = 0

    for i in teachers:
        for day_start in range(1, 50, 10):
            periods_day = sorted(set(p for j in classes for p in range(day_start, day_start + 10) if solution[i][j][p].varValue == 1))
            
            # Verificar períodos ociosos
            if periods_day:
                for p in range(day_start, day_start + 10):
                    if p not in periods_day:
                        previous_class = any(pp < p for pp in periods_day)
                        next_class = any(pp > p for pp in periods_day)
                        if previous_class and next_class:
                            idle_periods += 1
            
            # Verificar classes consecutivas
            for j in classes:
                for k in range(day_start, day_start + 9):
                    if solution[i][j][k].varValue == 1 and solution[i][j][k + 1].varValue == 1:
                        double_classes += 1
                        if k + 2 <= day_start + 9 and solution[i][j][k + 2].varValue == 1:
                            triple_classes += 1
    fo=1000*idle_periods - 500*double_classes + 5000*triple_classes;
    return idle_periods, double_classes, triple_classes, fo


def improve_solution(solution, teachers, classes):
    idle_periods, double_classes, triple_classes, fo = avaliar_solucao(solution)
    improved = True
    while improved:
        improved = False
        for teacher1 in teachers:
            for teacher2 in teachers:
                if teacher1 != teacher2:
                    for k in range(1, 49):  # Ajustado para evitar acessar k+1 no último período
                        for class1 in classes:
                            for class2 in classes:
                                if class1 != class2:
                                    # Verifica se as condições de troca são atendidas
                                    if (solution[teacher1][class1][k].varValue == 1 and
                                        solution[teacher1][class2][k + 1].varValue == 1 and
                                        solution[teacher2][class2][k].varValue == 1 and
                                        solution[teacher2][class1][k + 1].varValue == 1):
                                        # print(f"Trocando: teacher1={teacher1}, teacher2={teacher2}, class1={class1}, class2={class2}")
                                        # Troca as classes class1 e class2 nos professores teacher1 e teacher2
                                        solution[teacher1][class1][k].varValue = 0
                                        solution[teacher1][class2][k + 1].varValue = 0
                                        solution[teacher2][class2][k].varValue = 0
                                        solution[teacher2][class1][k + 1].varValue = 0
                                        solution[teacher1][class2][k].varValue = 1
                                        solution[teacher1][class1][k + 1].varValue = 1
                                        solution[teacher2][class1][k].varValue = 1
                                        solution[teacher2][class2][k + 1].varValue = 1

                                        # Avalia a solução após a troca
                                        new_idle_periods, new_double_classes, new_triple_classes,_ = avaliar_solucao(solution)

                                        # Verifica se a troca melhorou a solução
                                        if new_double_classes > double_classes or new_idle_periods < idle_periods:
                                        # if new_double_classes > double_classes or (new_double_classes == double_classes and new_idle_periods < idle_periods):
                                            idle_periods = new_idle_periods
                                            double_classes = new_double_classes
                                            triple_classes = new_triple_classes
                                            improved = True
                                        else:
                                            # Desfaz a troca se não melhorou a solução
                                            solution[teacher1][class2][k].varValue = 0
                                            solution[teacher1][class1][k + 1].varValue = 0
                                            solution[teacher2][class1][k].varValue = 0
                                            solution[teacher2][class2][k + 1].varValue = 0
                                            solution[teacher1][class1][k].varValue = 1
                                            solution[teacher1][class2][k + 1].varValue = 1
                                            solution[teacher2][class2][k].varValue = 1
                                            solution[teacher2][class1][k + 1].varValue = 1

    return solution, idle_periods, double_classes, triple_classes

# # Exemplo de uso
# improved_solution, new_idle_periods, new_double_classes, new_triple_classes = improve_solution(x, teachers, classes)
# print("Idle Periods after improvement:", new_idle_periods)
# print("Double Classes after improvement:", new_double_classes)
# print("Triple Classes after improvement:", new_triple_classes)



def optimize_teacher_assignment(double_classes_cost,idle_periods_cost,triple_classes_cost,teachers, classes_opt, classes, periods, costs, valores_x_opt, availability, teaching_load, ex_opt_groups, timeout=60):
    # Definir as variáveis de decisão
    M=1000000
    x_opt = LpVariable.dicts("x_opt", (teachers, classes_opt, periods), cat=LpBinary)
    idle_periods_var = LpVariable.dicts("idle count", (teachers, range(1, 51)), cat=LpBinary)
    z = LpVariable.dicts("z", (teachers, range(1, 51)), cat=LpBinary)
    y = LpVariable.dicts("y", (teachers, range(1, 51)), cat=LpBinary)
    pc = LpVariable.dicts("pc", (teachers, range(1, 51)), cat=LpBinary)
    nc = LpVariable.dicts("nc", (teachers, range(1, 51)), cat=LpBinary)
    ib = LpVariable.dicts("ib", (teachers, range(1, 51)), cat=LpBinary)
    is_busy = LpVariable.dicts("is_busy", (teachers, range(1, 51)), cat=LpBinary)
    previous_class = LpVariable.dicts("previous_class", (teachers, range(1, 51)), cat='Continuous')
    next_class = LpVariable.dicts("next_class", (teachers, range(1, 51)), cat='Continuous')
    double_classes_var = LpVariable.dicts("double_classes", (teachers, classes_opt, range(1, 50)), cat=LpBinary)
    triple_classes_var = LpVariable.dicts("triple_classes", (teachers, classes_opt, range(1, 49)), cat=LpBinary)

    # Problema de otimização
    prob = LpProblem("TeacherClassAssignment", LpMinimize)

    # Função objetivo (com múltiplos objetivos)
    prob += (
        lpSum(costs[(i, j, k)] * x_opt[i][j][k] for i in teachers for j in classes_opt for k in periods) 
        + double_classes_cost * lpSum(double_classes_var[i][j][k] for i in teachers for j in classes_opt for k in range(1, 50))
        + idle_periods_cost * lpSum(idle_periods_var[i][k] for i in teachers for k in periods)
        + triple_classes_cost * lpSum(triple_classes_var[i][j][k] for i in teachers for j in classes_opt for k in range(1, 49))
    )

    # Restrições
    # Disponibilidade dos professores
    for i in teachers:
        for k in periods:
            prob += (lpSum(x_opt[i][j][k] for j in classes_opt) + lpSum(valores_x_opt[(i, j, k)] for j in classes)) <= availability[(i, k)]

    # Cobertura das classes
    for j in classes_opt:
        for k in {1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 26, 31, 32, 33, 34, 35, 36, 41, 42, 43, 44, 45, 46}:
            prob += lpSum(x_opt[i][j][k] for i in teachers) == 1
        for k in {7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50}:
            prob += lpSum(x_opt[i][j][k] for i in teachers) <= 1

    # Carga horária máx_optima
    for i in teachers:
        for j in classes_opt:
            prob += lpSum(x_opt[i][j][k] for k in periods) == teaching_load[i][j]

    # Sem sobreposição de classes
    for i in teachers:
        for k in periods:
            prob += (lpSum(x_opt[i][j][k] for j in classes_opt) + lpSum(valores_x_opt[(i, j, k)] for j in classes)) <= 1

   # Grupos de professores que não podem dar aula no mesmo período
    for group in ex_opt_groups:
        for k in periods:
            prob += (lpSum(x_opt[i][j][k] for i in group for j in classes_opt) + lpSum(valores_x_opt[(i, j, k)] for i in group for j in classes if (i, j, k) in valores_x_opt)) <= 1

    # Evitar mais de 3 aulas no período da manhã por dia
    for i in teachers:
        if i == 47:
            continue
        for j in classes_opt:
            for day_start in range(1, 50, 10):  # Itera pelos dias começando em 1, 11, 21, 31, 41
                prob += lpSum(x_opt[i][j][day_start + k] for k in range(6)) <= 2

    # Variável auxiliar para pares de períodos consecutivos na mesma turma
    edu_fisica_pair = LpVariable.dicts("edu_fisica_pair", (teachers, classes_opt, periods), cat=LpBinary)
    
    # Restrição para garantir que na classe Educação Física só hajam aulas duplas
    for i in [12, 37]:
        for j in classes_opt:
            for k in range(1, 49):  # Ajuste para evitar os períodos finais
                if k % 10 not in {7, 8, 9, 10}:  # Evita os períodos específicos
                    prob += edu_fisica_pair[i][j][k] <= x_opt[i][j][k]
                    prob += edu_fisica_pair[i][j][k] <= x_opt[i][j][k + 1]
                    prob += edu_fisica_pair[i][j][k] >= x_opt[i][j][k] + x[i][j][k + 1] - 1
                    # Assegura que a soma das variáveis auxiliares edu_fisica_pair é igual a 1, 
                    # ou seja, garantindo que os períodos k e k+1 sejam ambos alocados ou não alocados para educação física
    
    for i in [12, 37]:
        for j in classes_opt:
            prob += lpSum(edu_fisica_pair[i][j][k] for k in [1,2,3,4,5,11,12,13,14,15,21,22,23,24,25,31,32,33,34,35,41,42,43,44,45,]) == teaching_load[i][j] / 2
    
    # Evitar que aulas de educação física ocorram nos períodos proibidos
    for i in [12, 37]:
        for j in classes_opt:
            for k in [7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50]:
                prob += x_opt[i][j][k] == 0

    
    for i in teachers:
        for k in range(1, 51):
            prob += nc[i][k] <= 1
            prob += pc[i][k] <= 1
            prob += ib[i][k] <= 1
            prob += z[i][k] <= 1
            prob += y[i][k] <= 1
            prob += idle_periods_var[i][k] <= 1
    
    for i in teachers:
        for day_start in range(1, 50, 10):
            for k in range(day_start + 1, day_start + 9):  # Evitando o primeiro período do dia
                # Definir se há aula no período atual
                prob += is_busy[i][k] == lpSum(x_opt[i][j][k] for j in classes_opt)
                prob += previous_class[i][k] == lpSum(x_opt[i][j][p] for j in classes_opt for p in range(day_start, k))
                prob += next_class[i][k] == lpSum(x_opt[i][j][p] for j in classes_opt for p in range(k + 1, day_start + 9))
    
                # Transformar as variáveis booleanas em binárias com limites superiores e inferiores
                prob += previous_class[i][k] <= M * pc[i][k]
                prob += previous_class[i][k] >= pc[i][k]
                prob += next_class[i][k] <= M * nc[i][k]
                prob += next_class[i][k] >= nc[i][k]
                prob += is_busy[i][k] <= M * ib[i][k]
                prob += is_busy[i][k] >= ib[i][k]
                prob += z[i][k] <= (pc[i][k] + (1 - is_busy[i][k])) / 2
                prob += z[i][k] >= (pc[i][k] + (1 - is_busy[i][k])) - 1
                prob += y[i][k] <= (z[i][k] + (nc[i][k])) / 2
                prob += y[i][k] >= (z[i][k] + (nc[i][k])) - 1
                prob += idle_periods_var[i][k] == y[i][k]
    # Restrições para contar double_classes
    for i in teachers:
        for j in classes_opt:
            for k in range(1, 50):
                prob += double_classes_var[i][j][k] >= x_opt[i][j][k] + x_opt[i][j][k + 1] - 1

    # Restrições para contar triple_classes
    for i in teachers:
        for j in classes_opt:
            for k in range(1, 49):
                prob += triple_classes_var[i][j][k] >= x_opt[i][j][k] + x_opt[i][j][k + 1] + x_opt[i][j][k + 2] - 2

    # Ex_optecução do solver com timeout
    solver = PULP_CBC_CMD(msg=True, timeLimit=timeout)  # Sem limite de tempo
    prob.solve(solver)

    # print("Status:", LpStatus[prob.status])
    # Atualizar valores das variáveis de decisão
    for i in teachers:
        for j in classes_opt:
            for k in range(1, 50):
                if x_opt[i][j][k].varValue is not None:
                    valores_x_opt[(i, j, k)] = x_opt[i][j][k].varValue 

    return valores_x_opt


def optimize_and_compare(all_classes, double_classes_cost, idle_periods_cost, triple_classes_cost, teachers, classesx2, classes_x2, periods, costs, valores_x0, availability, teaching_load, ex_groups, fo1):
    # Salvar os valores atuais
    current_values = {(i, j, k): x[i][j][k].varValue for i in teachers for j in all_classes for k in range(1, 50)}
    
    # Realizar a otimização
    valores_x1 = optimize_teacher_assignment(double_classes_cost, idle_periods_cost, triple_classes_cost, teachers, classesx2, classes_x2, periods, costs, valores_x0, availability, teaching_load, ex_groups)
    
    # Atualizar variáveis de decisão com os valores otimizados
    for i in teachers:
        for j in all_classes:
            for k in range(1, 50):
                x[i][j][k].varValue = valores_x1[(i, j, k)]
    improved_solution, _, _, _ = improve_solution(x, teachers, classes)
    # Atualizar variáveis de decisão com os valores otimizados
    for i in teachers:
        for j in all_classes:
            for k in range(1, 50):
                x[i][j][k].varValue = improved_solution[i][j][k].varValue
    # Avaliar a nova solução
    idle_periods, double_classes, triple_classes, fo2 = avaliar_solucao(x)
    
    # Mostrar o valor de FO na iteração atual
    print(f"FO anterior: {fo1}, FO atual: {fo2}")
    print("Idle Periods:", idle_periods)
    print("Double Classes:", double_classes)
    print("Triple Classes:", triple_classes)

    # Comparar a nova solução com a inicial e reverter se necessário
    if fo2 > fo1:
        # Restaurar os valores originais
        for (i, j, k) in current_values:
            x[i][j][k].varValue = current_values[(i, j, k)]
        print("Nova solução rejeitada.")
        return False, fo1, valores_x0  # Solução rejeitada
    else:
        fo1 = fo2  # Atualizar fo1 se a nova solução for melhor
        valores_x0 = valores_x1
        print("Nova solução aceita.")
        return True, fo1, valores_x0  # Solução aceita




Verifica se o problema relaxado tem solução

In [2]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value, PULP_CBC_CMD

tabela_prof = f'Horarios artificiais/horarios_embaralhado_linhas_1.csv'

# Carregar o arquivo CSV
dados = pd.read_csv(tabela_prof, sep=';', encoding='latin1')

def extract_availability(periods_str):
    if periods_str == '---------':
        return []
    available_periods = periods_str.split('-')
    return [int(p) for p in available_periods if p.isdigit()]

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
availability = {}
teachers = dados['No'].tolist()
periods = list(range(1, 51))  # 50 periods for each week

for i, row in dados.iterrows():
    teacher_id = row['No']
    for j, day in enumerate(days):
        day_period_start = j * 10 + 1
        available_periods = extract_availability(row[day])
        for period in range(1, 11):
            availability[(teacher_id, day_period_start + period - 1)] = 1 if period in available_periods else 0

# Definir as classes com base nas colunas do CSV
classes = ['6oA', '6oB', '7oA', '7oB', '8oA', '8oB','9oA', '9oB', '1oA', '1oB','2oA', '2oB', '3o']

# Modificar para gerar o t[i, j]
teaching_load = {}

# Verificar se as colunas de carga máxima existem no CSV
missing_columns = [col for col in classes if col not in dados.columns]

if missing_columns:
    raise KeyError(f"Missing columns in CSV: {', '.join(missing_columns)}")

# Carga horária em cada classe de cada professor
for i, row in dados.iterrows():
    teacher_id = row['No']
    teaching_load[teacher_id] = {}
    for cls in classes:
        teaching_load[teacher_id][cls] = row[cls]

# Definir os custos
costs = {}
for i, row in dados.iterrows():
    teacher_id = row['No']
    for cls in classes:
        for day in range(5):
            for period in range(1, 11):
                actual_period = day * 10 + period
                cost = 1 if period <= 6 else 1000 if period == 7 else 5000 if period == 8 else 10000 if period == 9 else 40000
                costs[(teacher_id, cls, actual_period)] = cost

# Variáveis de decisão
x = LpVariable.dicts("x", (teachers, classes, periods), cat=LpBinary)

# Problema de otimização
prob = LpProblem("TeacherClassAssignment", LpMinimize)

# Função objetivo
prob += lpSum(costs[(i, j, k)] * x[i][j][k] for i in teachers for j in classes for k in periods)

# Restrições

# Disponibilidade dos professores
for i in teachers:
    for k in periods:
        prob += lpSum(x[i][j][k] for j in classes) <= availability[(i, k)]

# Cobertura das classes
for j in classes:
    for k in {1,2,3,4,5,6,11,12,13,14,15,16,21,22,23,24,25,26,31,32,33,34,35,36,41,42,43,44,45,46}:
        prob += lpSum(x[i][j][k] for i in teachers) == 1
    for k in {7,8,9,10,17,18,19,20,27,28,29,30,37,38,39,40,47,48,49,50}:
        prob += lpSum(x[i][j][k] for i in teachers) <= 1

# Carga horária máxima
for i in teachers:
    for j in classes:
        prob += lpSum(x[i][j][k] for k in periods) == teaching_load[i][j]

# Sem sobreposição de classes
for i in teachers:
    for k in periods:
        prob += lpSum(x[i][j][k] for j in classes) <= 1

# Grupos de professores que não podem dar aula no mesmo período
exclusive_groups = [
    #[4, 44],
    [45, 22, 25],
    [46, 38, 31],
    [32, 33]
]


for group in exclusive_groups:
    for k in periods:
        prob += lpSum(x[i][j][k] for i in group for j in classes) <= 1

# Evitar mais de 3 aulas no período da manhã por dia
for i in teachers:
    if i == 47:
        continue
    for j in classes:
        for day_start in range(1, 50, 10):  # Itera pelos dias começando em 1, 11, 21, 31, 41
            prob += lpSum(x[i][j][day_start + k] for k in range(6)) <= 2

# Conjunto dos períodos a serem evitados para educação física
periods_to_avoid = {7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50}

# Variável auxiliar para pares de períodos consecutivos na mesma turma
edu_fisica_pair = LpVariable.dicts("edu_fisica_pair", (teachers, classes, periods), cat=LpBinary)

# Restrição para garantir que na classe Educação Física só hajam aulas duplas
for i in [12, 37]:
    for j in classes:
        for k in range(1, 49):  # Ajuste para evitar os períodos finais
            if k % 10 not in {7, 8, 9, 10}:  # Evita os períodos específicos
                prob += edu_fisica_pair[i][j][k] <= x[i][j][k]
                prob += edu_fisica_pair[i][j][k] <= x[i][j][k + 1]
                prob += edu_fisica_pair[i][j][k] >= x[i][j][k] + x[i][j][k + 1] - 1
                # Assegura que a soma das variáveis auxiliares edu_fisica_pair é igual a 1, 
                # ou seja, garantindo que os períodos k e k+1 sejam ambos alocados ou não alocados para educação física

for i in [12, 37]:
    for j in classes:
        prob += lpSum(edu_fisica_pair[i][j][k] for k in [1,2,3,4,5,11,12,13,14,15,21,22,23,24,25,31,32,33,34,35,41,42,43,44,45,]) == teaching_load[i][j] / 2

# Evitar que aulas de educação física ocorram nos períodos proibidos
for i in [12, 37]:
    for j in classes:
        for k in [7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50]:
            prob += x[i][j][k] == 0

# for i in [12,37]:
#     for j in classes:
#         for k in range(1, 50, 2):  # Itere apenas sobre os períodos ímpares para garantir a continuidade
#             prob += x[i][j][k] == x[i][j][k + 1]

# # # # Restrição para garantir que na classe '3o' só hajam aulas duplas
# for i in teachers:
#     for k in range(1, 50, 2):  # Itere apenas sobre os períodos ímpares para garantir a continuidade
#         prob += x[i]['3o'][k] == x[i]['3o'][k + 1]

# Execução do solver com timeout
#solver = PULP_CBC_CMD(timeLimit=30, msg=True)  # 'timeLimit' define o tempo máximo
solver = PULP_CBC_CMD(msg=True)  # Sem limite de tempo
prob.solve(solver)

print("Status:", LpStatus[prob.status])

# Criar um dicionário para armazenar os valores das variáveis de decisão
valores_x = {}

# Iterar sobre as combinações de professores, disciplinas e períodos
for i in teachers:
    for j in ['6oA', '6oB', '7oA', '7oB','8oA', '8oB','9oA', '9oB', '1oA', '1oB', '2oA', '2oB', '3o']:
        for k in range(1, 51):
            # Verificar se a variável de decisão está definida e obter o valor
            if x[i][j][k].varValue is not None:
                valores_x[(i, j, k)] = x[i][j][k].varValue

Status: Optimal


In [3]:
# Avalia a solução inicial
idle_periods_ORIGINAL, double_classes_ORIGINAL, triple_classes_ORIGINAL, fo_ORIGINAL = avaliar_solucao(x)
print("Idle Periods:", idle_periods_ORIGINAL)
print("Double Classes:", double_classes_ORIGINAL)
print("Triple Classes:", triple_classes_ORIGINAL)
print("Triple Classes:", fo_ORIGINAL)


Idle Periods: 80
Double Classes: 44
Triple Classes: 2
Triple Classes: 68000


In [4]:
# Gera um excel colorido da versão inicial
generate_and_export_timetable(teachers, classes, periods, x, dados, 'timetable_colored_0.xlsx')

Arquivo Excel salvo como timetable_colored_0.xlsx


In [5]:
# Fix and Optimize Heuristic
# Inicializar variáveis
import time
# Iniciar a contagem de tempo
start_time = time.time()

# Inicializar variáveis
valores_x0 = valores_x
idle_periods, double_classes, triple_classes, fo1 = avaliar_solucao(x)
print("fo1:", fo1)

# Definir todas as turmas
all_classes = ['6oA', '6oB', '7oA', '7oB', '8oA', '8oB', '9oA', '9oB', '1oA', '1oB', '2oA', '2oB', '3o']

double_classes_cost = -500
idle_periods_cost = 1000
triple_classes_cost = 5000

# Iterar até 20 iterações ou até todas as soluções serem rejeitadas
max_iterations = 20
iteration = 0
all_rejected = False

while iteration < max_iterations and not all_rejected:
    iteration += 1
    all_rejected = True  # Suponha que todas as soluções serão rejeitadas nesta iteração
    print(f"\nIteração {iteration}:")

    for i in range(0, len(all_classes), 2):
        classesx2 = all_classes[i:i+2]
        classes_x2 = [cls for cls in all_classes if cls not in classesx2]
        ex_groups = [[45, 22, 25], [46, 38, 31], [32, 33]]
        print(f"Otimizando para turmas: {classesx2}")
        accepted, fo1, valores_x0 = optimize_and_compare(all_classes,double_classes_cost,idle_periods_cost,triple_classes_cost,teachers, classesx2, classes_x2, periods, costs, valores_x0, availability, teaching_load, ex_groups, fo1)
        if accepted:
            all_rejected = False  # Se alguma solução foi aceita, não interrompa ainda

    if all_rejected:
        print(f"Ponto ótimo alcançado em {iteration} iterações.")
        break

if iteration == max_iterations:
    print("Número limite de iterações atingido.")

improved_solution, new_idle_periods, new_double_classes, new_triple_classes = improve_solution(x, teachers, classes)

# Parar a contagem de tempo
end_time = time.time()

# Calcular o tempo total de execução
total_time = end_time - start_time
print(f"Tempo total de execução: {total_time} segundos")


fo1: 68000

Iteração 1:
Otimizando para turmas: ['6oA', '6oB']
FO anterior: 68000, FO atual: 68500
Idle Periods: 82
Double Classes: 47
Triple Classes: 2
Nova solução rejeitada.
Otimizando para turmas: ['7oA', '7oB']
FO anterior: 68000, FO atual: 66000
Idle Periods: 80
Double Classes: 48
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['8oA', '8oB']
FO anterior: 66000, FO atual: 56500
Idle Periods: 74
Double Classes: 55
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['9oA', '9oB']
FO anterior: 56500, FO atual: 58000
Idle Periods: 76
Double Classes: 56
Triple Classes: 2
Nova solução rejeitada.
Otimizando para turmas: ['1oA', '1oB']
FO anterior: 56500, FO atual: 50000
Idle Periods: 68
Double Classes: 56
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['2oA', '2oB']
FO anterior: 50000, FO atual: 47000
Idle Periods: 67
Double Classes: 60
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['3o']
FO anterior: 47000, FO atual: 46500
Idl

In [6]:
# Avalia a solução final
idle_periods, double_classes, triple_classes, fo = avaliar_solucao(improved_solution)
print("Idle Periods:", idle_periods)
print("Double Classes:", double_classes)
print("Triple Classes:", triple_classes)
print("FO:", fo)

improved_solution, new_idle_periods, new_double_classes, new_triple_classes = improve_solution(x, teachers, classes)

Idle Periods: 56
Double Classes: 59
Triple Classes: 2
FO: 36500


In [7]:
# Gera a grade de horários colorida
generate_and_export_timetable(teachers, classes, periods, improved_solution, dados, 'timetable_coloredFO_final.xlsx')

Arquivo Excel salvo como timetable_coloredFO_final.xlsx


Medição de dados

In [ ]:
import time
import pandas as pd
# Listas para armazenar os resultados
total_time = []
media_tempo_iteracao = []
antes_idle_periods = []
antes_double_classes = []
antes_triple_classes = []
antes_fo = []
depois_idle_periods = []
depois_double_classes = []
depois_triple_classes = []
depois_fo = []
iterations = []  # Lista para armazenar o número da iteração
optimalmsg =[]
start_time_ALLPROGRAM = time.time()
for n in range(1, 101):
    print(n)
    # Iniciar contagem de tempo total
    start_time_total = time.time()
    tabela_prof = f'Horarios artificiais/horarios_embaralhado_linhas_{n}.csv'
    # tabela_prof = 'tabela_profDados.csv'
    
    import pandas as pd
    from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value, PULP_CBC_CMD
    
    
    
    # Carregar o arquivo CSV
    dados = pd.read_csv(tabela_prof, sep=';', encoding='latin1')
    
    def extract_availability(periods_str):
        if periods_str == '---------':
            return []
        available_periods = periods_str.split('-')
        return [int(p) for p in available_periods if p.isdigit()]
    
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    availability = {}
    teachers = dados['No'].tolist()
    periods = list(range(1, 51))  # 50 periods for each week
    
    for i, row in dados.iterrows():
        teacher_id = row['No']
        for j, day in enumerate(days):
            day_period_start = j * 10 + 1
            available_periods = extract_availability(row[day])
            for period in range(1, 11):
                availability[(teacher_id, day_period_start + period - 1)] = 1 if period in available_periods else 0
    
    # Definir as classes com base nas colunas do CSV
    classes = ['6oA', '6oB', '7oA', '7oB', '8oA', '8oB','9oA', '9oB', '1oA', '1oB','2oA', '2oB', '3o']
    
    # Modificar para gerar o t[i, j]
    teaching_load = {}
    
    # Verificar se as colunas de carga máxima existem no CSV
    missing_columns = [col for col in classes if col not in dados.columns]
    
    if missing_columns:
        raise KeyError(f"Missing columns in CSV: {', '.join(missing_columns)}")
    
    # Carga horária em cada classe de cada professor
    for i, row in dados.iterrows():
        teacher_id = row['No']
        teaching_load[teacher_id] = {}
        for cls in classes:
            teaching_load[teacher_id][cls] = row[cls]
    
    # Definir os custos
    costs = {}
    for i, row in dados.iterrows():
        teacher_id = row['No']
        for cls in classes:
            for day in range(5):
                for period in range(1, 11):
                    actual_period = day * 10 + period
                    cost = 1 if period <= 6 else 1000 if period == 7 else 5000 if period == 8 else 10000 if period == 9 else 40000
                    costs[(teacher_id, cls, actual_period)] = cost
    
    # Variáveis de decisão
    x = LpVariable.dicts("x", (teachers, classes, periods), cat=LpBinary)
    
    # Problema de otimização
    prob = LpProblem("TeacherClassAssignment", LpMinimize)
    
    # Função objetivo
    prob += lpSum(costs[(i, j, k)] * x[i][j][k] for i in teachers for j in classes for k in periods)
    
    # Restrições
    
    # Disponibilidade dos professores
    for i in teachers:
        for k in periods:
            prob += lpSum(x[i][j][k] for j in classes) <= availability[(i, k)]
    
    # Cobertura das classes
    for j in classes:
        for k in {1,2,3,4,5,6,11,12,13,14,15,16,21,22,23,24,25,26,31,32,33,34,35,36,41,42,43,44,45,46}:
            prob += lpSum(x[i][j][k] for i in teachers) == 1
        for k in {7,8,9,10,17,18,19,20,27,28,29,30,37,38,39,40,47,48,49,50}:
            prob += lpSum(x[i][j][k] for i in teachers) <= 1
    
    # Carga horária máxima
    for i in teachers:
        for j in classes:
            prob += lpSum(x[i][j][k] for k in periods) == teaching_load[i][j]
    
    # Sem sobreposição de classes
    for i in teachers:
        for k in periods:
            prob += lpSum(x[i][j][k] for j in classes) <= 1
    
    # Grupos de professores que não podem dar aula no mesmo período
    exclusive_groups = [
        #[4, 44],
        [45, 22, 25],
        [46, 38, 31],
        [32, 33]
    ]
    
    
    for group in exclusive_groups:
        for k in periods:
            prob += lpSum(x[i][j][k] for i in group for j in classes) <= 1
    
    # Evitar mais de 3 aulas no período da manhã por dia
    for i in teachers:
        if i == 47:
            continue
        for j in classes:
            for day_start in range(1, 50, 10):  # Itera pelos dias começando em 1, 11, 21, 31, 41
                prob += lpSum(x[i][j][day_start + k] for k in range(6)) <= 2
    
    # Conjunto dos períodos a serem evitados para educação física
    periods_to_avoid = {7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50}
    
    # Variável auxiliar para pares de períodos consecutivos na mesma turma
    edu_fisica_pair = LpVariable.dicts("edu_fisica_pair", (teachers, classes, periods), cat=LpBinary)
    
    # Restrição para garantir que na classe Educação Física só hajam aulas duplas
    for i in [12, 37]:
        for j in classes:
            for k in range(1, 49):  # Ajuste para evitar os períodos finais
                if k % 10 not in {7, 8, 9, 10}:  # Evita os períodos específicos
                    prob += edu_fisica_pair[i][j][k] <= x[i][j][k]
                    prob += edu_fisica_pair[i][j][k] <= x[i][j][k + 1]
                    prob += edu_fisica_pair[i][j][k] >= x[i][j][k] + x[i][j][k + 1] - 1
                    # Assegura que a soma das variáveis auxiliares edu_fisica_pair é igual a 1, 
                    # ou seja, garantindo que os períodos k e k+1 sejam ambos alocados ou não alocados para educação física
    
    for i in [12, 37]:
        for j in classes:
            prob += lpSum(edu_fisica_pair[i][j][k] for k in [1,2,3,4,5,11,12,13,14,15,21,22,23,24,25,31,32,33,34,35,41,42,43,44,45,]) == teaching_load[i][j] / 2
    
    # Evitar que aulas de educação física ocorram nos períodos proibidos
    for i in [12, 37]:
        for j in classes:
            for k in [7, 8, 9, 10, 17, 18, 19, 20, 27, 28, 29, 30, 37, 38, 39, 40, 47, 48, 49, 50]:
                prob += x[i][j][k] == 0
    
    # for i in [12,37]:
    #     for j in classes:
    #         for k in range(1, 50, 2):  # Itere apenas sobre os períodos ímpares para garantir a continuidade
    #             prob += x[i][j][k] == x[i][j][k + 1]
    
    # # # # Restrição para garantir que na classe '3o' só hajam aulas duplas
    # for i in teachers:
    #     for k in range(1, 50, 2):  # Itere apenas sobre os períodos ímpares para garantir a continuidade
    #         prob += x[i]['3o'][k] == x[i]['3o'][k + 1]
    
    # Execução do solver com timeout
    #solver = PULP_CBC_CMD(timeLimit=30, msg=True)  # 'timeLimit' define o tempo máximo
    solver = PULP_CBC_CMD(msg=True)  # Sem limite de tempo
    prob.solve(solver)
    optimalmsg.append(LpStatus[prob.status])
    # Criar um dicionário para armazenar os valores das variáveis de decisão
    valores_x = {}
    
    # Iterar sobre as combinações de professores, disciplinas e períodos
    for i in teachers:
        for j in ['6oA', '6oB', '7oA', '7oB','8oA', '8oB','9oA', '9oB', '1oA', '1oB', '2oA', '2oB', '3o']:
            for k in range(1, 51):
                # Verificar se a variável de decisão está definida e obter o valor
                if x[i][j][k].varValue is not None:
                    valores_x[(i, j, k)] = x[i][j][k].varValue

    
    # Inicializar variáveis
    valores_x0 = valores_x
    
    # Definir todas as turmas
    all_classes = ['6oA', '6oB', '7oA', '7oB', '8oA', '8oB', '9oA', '9oB', '1oA', '1oB', '2oA', '2oB', '3o']
    
    double_classes_cost = -500
    idle_periods_cost = 1000
    triple_classes_cost = 5000
    
    ex_groups = [[45, 22, 25], [46, 38, 31], [32, 33]]
    
    # Registrar estado antes da heurística
    idle_periods, double_classes, triple_classes, fo1 = avaliar_solucao(x)
    antes_idle_periods.append(idle_periods)
    antes_double_classes.append(double_classes)
    antes_triple_classes.append(triple_classes)
    antes_fo.append(fo1)
    

    print("fo1:", fo1)
    
    # Iterar até 20 iterações ou até todas as soluções serem rejeitadas
    max_iterations = 20
    iteration = 0
    all_rejected = False
    
    # Lista para armazenar tempos de cada iteração
    tempos_iteracoes = []
    
    while iteration < max_iterations and not all_rejected:
        iteration += 1
        all_rejected = True  # Suponha que todas as soluções serão rejeitadas nesta iteração
        print(f"\nIteração {iteration}:")
    
        for i in range(0, len(all_classes), 2):
            classesx2 = all_classes[i:i+2]
            classes_x2 = [cls for cls in all_classes if cls not in classesx2]
            print(f"Otimizando para turmas: {classesx2}")
    
            # Iniciar contagem de tempo para a iteração
            start_time_iter = time.time()
    
            accepted, fo1, valores_x0 = optimize_and_compare(all_classes, double_classes_cost, idle_periods_cost,
                                                             triple_classes_cost, teachers, classesx2, classes_x2,
                                                             periods, costs, valores_x0, availability,
                                                             teaching_load, ex_groups, fo1)
    
            # Finalizar contagem de tempo para a iteração
            end_time_iter = time.time()
            tempo_iteracao = end_time_iter - start_time_iter
            tempos_iteracoes.append(tempo_iteracao)
    
            if accepted:
                all_rejected = False  # Se alguma solução foi aceita, não interrompa ainda

        if all_rejected:
            print(f"Ponto ótimo alcançado em {iteration} iterações.")
            break
    
    if iteration == max_iterations:
        print("Número limite de iterações atingido.")
    
    # Registrar estado depois das iterações
    idle_periods, double_classes, triple_classes, fo1 = avaliar_solucao(x)
    depois_idle_periods.append(idle_periods)
    depois_double_classes.append(double_classes)
    depois_triple_classes.append(triple_classes)
    depois_fo.append(fo1)
    
    # Calcular o tempo total de execução
    end_time_total = time.time()
    total_time.append(end_time_total - start_time_total)
    media_tempo_iteracao.append(sum(tempos_iteracoes) / len(tempos_iteracoes) if tempos_iteracoes else 0)
    iterations.append(iteration)  # Armazenar o valor de iteration


# DataFrame com todos os dados coletados
data = {
    'Optimal': optimalmsg,
    'Iteration':iterations,
    'Total time (seconds)': total_time,
    'Average Iteration Time (seconds)': media_tempo_iteracao,
    'Idle Periods Before': antes_idle_periods,
    'Idle Periods After': depois_idle_periods,
    'Double Classes After': depois_double_classes,
    'Double Classes Before': antes_double_classes,
    'Triple Classes Before': antes_triple_classes,
    'Triple Classes After': depois_triple_classes,
    'Objective Function Before': antes_fo,
    'Objective Function After': depois_fo
}
# Criar o DataFrame
df = pd.DataFrame(data)

# Calcular as melhorias
df['Idle Periods Improvement'] = (1 - (df['Idle Periods After'] / df['Idle Periods Before'])) * 100
df['Double Classes Improvement'] = ((df['Double Classes After'] / df['Double Classes Before']) - 1) * 100
df['Triple Classes Improvement'] = (1 - (df['Triple Classes After'] / df['Triple Classes Before'])) * 100
df['Objective Function Improvement'] = (1-(df['Objective Function After'] / df['Objective Function Before']) ) * 100

# Formatar as melhorias com duas casas decimais e o símbolo de porcentagem
df['Idle Periods Improvement'] = df['Idle Periods Improvement'].apply(lambda x: f"{x:.2f}%")
df['Double Classes Improvement'] = df['Double Classes Improvement'].apply(lambda x: f"{x:.2f}%")
df['Triple Classes Improvement'] = df['Triple Classes Improvement'].apply(lambda x: f"{x:.2f}%")
df['Objective Function Improvement'] = df['Objective Function Improvement'].apply(lambda x: f"{x:.2f}%")

# Remover colunas indesejadas
df = df[['Optimal','Iteration', 'Total time (seconds)', 'Average Iteration Time (seconds)', 
         'Idle Periods Before', 'Idle Periods After', 'Idle Periods Improvement',
         'Double Classes Before', 'Double Classes After', 'Double Classes Improvement',
         'Triple Classes Before', 'Triple Classes After', 'Triple Classes Improvement',
         'Objective Function Before', 'Objective Function After', 'Objective Function Improvement']]

# Nome do arquivo de saída
# output = f"Dados_sobre_tabela_artifical{n+2}.xlsx"
output = f"Dados_sobre_tabela_real_Consertado{n+6}.xlsx"
# # Criar DataFrame
# df = pd.DataFrame(data)
# Salvar o DataFrame em um arquivo Excel
df.to_excel(output, index=False)

# Calcular o tempo total de execução
end_timeTotal = time.time()
total_time = end_timeTotal - start_time_ALLPROGRAM
print(f"Tempo total de execução: {total_time} segundos")



1
fo1: 68000

Iteração 1:
Otimizando para turmas: ['6oA', '6oB']
FO anterior: 68000, FO atual: 68500
Idle Periods: 82
Double Classes: 47
Triple Classes: 2
Nova solução rejeitada.
Otimizando para turmas: ['7oA', '7oB']
FO anterior: 68000, FO atual: 66000
Idle Periods: 80
Double Classes: 48
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['8oA', '8oB']
FO anterior: 66000, FO atual: 56500
Idle Periods: 74
Double Classes: 55
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['9oA', '9oB']
FO anterior: 56500, FO atual: 58000
Idle Periods: 76
Double Classes: 56
Triple Classes: 2
Nova solução rejeitada.
Otimizando para turmas: ['1oA', '1oB']
FO anterior: 56500, FO atual: 50000
Idle Periods: 68
Double Classes: 56
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['2oA', '2oB']
FO anterior: 50000, FO atual: 47000
Idle Periods: 67
Double Classes: 60
Triple Classes: 2
Nova solução aceita.
Otimizando para turmas: ['3o']
FO anterior: 47000, FO atual: 46500
I